In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os
import json
import pandas as pd
import numpy as np
import statistics

In [13]:
df = pd.read_csv('data/ownerdf.csv')

In [14]:
df.head(3)

,title,owners_median,positive,negative,num_reviews,ccu,median_2weeks,median_forever,average_2weeks,average_forever,...,Electronic Arts,Paradox Interactive,SEGA,Sokpop Collective,Square Enix,THQ Nordic,Ubisoft,Winged Cloud,other,tinyBuild
0,OUTRIDERS,1500000.0,26922,14940,36837,23037,279,1600,585,2074,...,0,0,0,0,1,0,0,0,0,0
1,It Takes Two,750000.0,20205,877,15688,11513,425,753,371,670,...,1,0,0,0,0,0,0,0,0,0
2,Valheim,15000000.0,222551,8466,222855,64931,435,1113,820,2205,...,0,0,0,0,0,0,0,0,1,0


In [15]:
df.shape

(5188, 92)

In [17]:
list(df.title)[:3]

['OUTRIDERS', 'It Takes Two', 'Valheim']

In [23]:
chromedriver = "/Applications/chromedriver" 
os.environ["webdriver.chrome.driver"] = chromedriver

url = 'https://steamspy.com/search.php'

driver = webdriver.Chrome(chromedriver)
driver.get(url)



In [22]:
driver.quit()

# Game Specific Scraping

In [24]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [31]:
#number of owners
soup.find('strong',text='Owners').next_sibling

': 15,463,000 '